Read the ansible all.yml file across multiple environments.

In [ ]:
%reset -f

In [ ]:
import argparse
import sys
import yaml
import re
import os
import sqlite3

class PathToVarFile:
    '''
    Return the path to the ansible all.yml file.
    '''
    def __init__(self, ts='.', prod='../ansible'):
        self.tsansible = ts
        self.prodansible = prod
        
    def path(self, env_name):
        if env_name in ['env1', 'env2']:
            return os.path.join(self.prodansible, 'environment', env_name, 'group_vars', 'all.yml')
        else:
            return os.path.join(self.tsansible, 'environment', env_name, 'group_vars', 'all.yml')

class ListValues:
    def __init__(self):
        self.filter = None
        self.prodansible_root = '../ansible/'
        self.tsansible_root = './'
        self.env = list()
    
    def setup(self):
        parser = argparse.ArgumentParser(description="List ansible variable values")

        # Only one argument in this group can be used
        # group = parser.add_mutually_exclusive_group()

        parser.add_argument(
            "-v", 
            "--verbose",
            action="store_true",
            help="Boolean flag"
            )

        # --argument value
        # value stored in args.argument
        parser.add_argument("--filter", dest='filter', action="store", help="Report keys matching regular expression")
        parser.add_argument("--env", '-e', dest='env', action="append", help="Environments to report. Repeat option for each env.")
        parser.add_argument("--prodansible", dest='prodansible', default=self.prodansible_root, action="store", help="Path to prodansible directory")
        parser.add_argument("--tsansible", dest='tsansible', default=self.tsansible_root, action="store", help="Path to tsansible directory")
        # results of parsing arguments
        args = parser.parse_args()

        self.filter = args.filter
        self.tsansible_root = args.tsansible
        self.prodansible_root = args.prodansible
        self.env = args.env
        if not self.env:
            self.env = ['env0', 'env1', 'env2']
            
    def show_args(self):
        print("--filter {}".format(self.filter))
        for entry in self.env:
            print("--env {}".format(entry))
            
    def walk_struct(self, struct, keys=[]):
        '''
        Generate SQL entries from YAML -> JSON struct

        Args:
            struct (json converted to python): Dict of dicts with possible lists. Created by converted YAML to JSON/Python.

            keys (list): List of dict key names. 
                keys[0]: first level dict 
                keys[1]: second level dict
                etc

        Yield:
            [long_key, value] 
            Where long_key is the '.' joined dict of dict name of each key in struct.
        '''

        if isinstance(struct, dict):
            for k in sorted(struct):
                v = struct[k]
                if isinstance(v, dict):
                    # new dict, descend
                    for result in self.walk_struct(v, keys=keys[:] + [str(k)]):
                        yield(result)
                elif isinstance(v, list):
                    # new list, ignore
                    pass
                else:
                    # yield [key.name1.name2, value]
                    yield(['.'.join(keys + [str(k)]), str(v)])

        else:
            # don't know how to do this
            pass

    def walk_values(self, fn, environment='', filter_re='^.*$'):
        '''
        Generate an entry suitable for inserting into the SQL key_value table.

        Args:
            fn (string): Ansible YAML values file filename
            environment (string): Human string naming environment
            filter_re (string): Python regular expression of items to keep. Non-matches are discarded. 

        Yields:
            [environment, key, value]
                environment (string): ENV1, ENV2 etc.
                key (strig): '.' joined dict of dict... string
                value: Ansible value of that key
        '''

        regex = re.compile(filter_re)

        fh  = open(fn, mode="r")
        docs = yaml.load_all(fh)
        env_vars = docs.next()

        for entry in self.walk_struct(env_vars):
            if regex.match(entry[0]):
                yield( [environment] + entry )

    def show_env(self, fn, name=''):
        fh  = open(fn, mode="r")
        docs = yaml.load_all(fh)
        env_vars = docs.next()

        for entry in walk_struct(env_vars):
            print ': '.join(entry)

    def envs(self):
        for i,e in enumerate(self.env):
            yield(e,i)
                    
    def generate_report(self):
        connection = sqlite3.connect(':memory:')
        cursor = connection.cursor()
        
        # Environments table
        connection.execute('''
            CREATE TABLE env 
                (
                env_id TEXT NOT NULL,
                rank   INT,
                PRIMARY KEY(env_id)
                );
            ''')

        # values table
        connection.execute('''
            CREATE TABLE key_value
                (
                env_id TEXT NOT NULL,
                name   TEXT NOT NULL,
                value  TEXT,
                PRIMARY KEY(env_id, name)
                );
            ''')

        cursor.executemany("insert into env values (?,?)", self.envs())

        # Load each variable file into the key_value table
        p = PathToVarFile(ts=self.tsansible_root, prod=self.prodansible_root)
        
        for env in self.env:
            cursor.executemany("insert into key_value values (?,?,?)", 
               self.walk_values(fn=p.path(env), environment=env, filter_re=self.filter))
            
        connection.commit()
        
        cursor.execute("select env_id, rank from env")
        # print cursor.fetchall()        

        # Create a unique list of keys across all environments

        # ENV | KEY
        connection.execute('''
            CREATE TABLE env_keys
             AS SELECT e.env_id, e.rank, k.name 
                FROM env e
                JOIN (SELECT distinct name from key_value) AS k;
            ''')
        
        cursor.execute('''
            SELECT k.name, k.env_id, v.value
            FROM env_keys k
            LEFT OUTER JOIN key_value v
            USING (env_id, name) 
            ORDER BY k.name, k.rank;
            ''')
        
        last_key = None
        for key, env, value in cursor.fetchall():
            if not key == last_key:
                print(key)
                last_key = key
            print("  {}: '{}'".format(env,value))
            
        connection.close()

In [ ]:
if __name__ == '__main__':
    app = ListValues()
    sys.argv = ['', 
        '--filter', '^ecommerce\..*', 
        '--env', 'env1',
        '--env', 'env2',
       ]

    app.setup()
    app.show_args()
    app.generate_report()

Code pieces

In [ ]:
p = PathToVarFile(ts='/Users/brian.jones/Desktop/CODE/git/tsansible', prod='/Users/brian.jones/Desktop/CODE/git/ansible')
print p.path('ite')
print p.path('ord')

In [ ]:

env1 = '/Users/brian.jones/Desktop/CODE/git/ansible/environment/env1/group_vars/all.yml'
env2 = '/Users/brian.jones/Desktop/CODE/git/ansible/environment/env2/group_vars/all.yml'

# limit output to
report_filter = '^ecommerce\..*'

In [ ]:
def walk_struct(struct, keys=[]):
    '''
    Generate SQL entries from YAML -> JSON struct
    
    Args:
        struct (json converted to python): Dict of dicts with possible lists. Created by converted YAML to JSON/Python.
        
        keys (list): List of dict key names. 
            keys[0]: first level dict 
            keys[1]: second level dict
            etc
    
    Yield:
        [long_key, value] 
        Where long_key is the '.' joined dict of dict name of each key in struct.
    '''
    
    if isinstance(struct, dict):
        for k in sorted(struct):
            v = struct[k]
            if isinstance(v, dict):
                # new dict, descend
                for result in walk_struct(v, keys=keys[:] + [str(k)]):
                    yield(result)
            elif isinstance(v, list):
                # new list, ignore
                pass
            else:
                # yield [key.name1.name2, value]
                yield(['.'.join(keys + [str(k)]), str(v)])
                
    else:
        # don't know how to do this
        pass

def walk_values(fn, environment='', filter_re='^.*$'):
    '''
    Generate an entry suitable for inserting into the SQL key_value table.
    
    Args:
        fn (string): Ansible YAML values file filename
        environment (string): Human string naming environment
        filter_re (string): Python regular expression of items to keep. Non-matches are discarded. 
        
    Yields:
        [environment, key, value]
            environment (string): ENV1, ENV2 etc.
            key (strig): '.' joined dict of dict... string
            value: Ansible value of that key
    '''
    
    filter = re.compile(filter_re)
    
    fh  = open(fn, mode="r")
    docs = yaml.load_all(fh)
    env_vars = docs.next()

    for entry in walk_struct(env_vars):
        if filter.match(entry[0]):
            yield( [environment] + entry )

def show_env(fn, name=''):
    fh  = open(fn, mode="r")
    docs = yaml.load_all(fh)
    env_vars = docs.next()

    for entry in walk_struct(env_vars):
        print ': '.join(entry)

# Sqlite3

In [ ]:
import sqlite3

In [ ]:
connection = sqlite3.connect(':memory:')
cursor = connection.cursor()

In [ ]:
# Environments table

connection.execute('''
CREATE TABLE env 
    (
    env_id TEXT NOT NULL,
    rank   INT,
    PRIMARY KEY(env_id)
    );
''')

# values table

connection.execute('''
CREATE TABLE key_value
    (
    env_id TEXT NOT NULL,
    name   TEXT NOT NULL,
    value  TEXT,
    PRIMARY KEY(env_id, name)
    );
''')

In [ ]:
cursor.executemany("insert into env values (?,?)", envs())

In [ ]:
# Insert rows
cursor.executemany("insert into key_value values (?,?,?)", walk_values(fn=env1, environment='ENV1', filter_re=report_filter))
cursor.executemany("insert into key_value values (?,?,?)", walk_values(fn=env2, environment='ENV2', filter_re=report_filter))

connection.commit()

In [ ]:
cursor.execute("select env_id, rank from env")
print cursor.fetchall()

In [ ]:
# Create a unique list of keys across all environments

# ENV | KEY

connection.execute('''
CREATE TABLE env_keys
 AS SELECT e.env_id, e.rank, k.name 
    FROM env e
    JOIN (SELECT distinct name from key_value) AS k;
''')

In [ ]:
cursor.execute('''
SELECT k.name, k.env_id, v.value
FROM env_keys k
LEFT OUTER JOIN key_value v
USING (env_id, name) 
ORDER BY k.name, k.rank;
''')

In [ ]:
last_key = None
for key, env, value in cursor.fetchall():
    if not key == last_key:
        print(key)
        last_key = key
    print("  {}: '{}'".format(env,value))

In [ ]:
connection.close()

Legacy code

In [ ]:
def show_values(fn, env_name):
    fh  = open(fn, mode="r")
    docs = yaml.load_all(fh)
    
    doc = docs.next()
    print
    print("{}:".format(env_name))
    print
    print('jvm Xms       : {}'.format(doc['ecommerce']['jvm']['Xms']))
    print('jvm Xmx       : {}'.format(doc['ecommerce']['jvm']['Xmx']))
    print('ro initialsize: {}'.format(doc['ecommerce']['db']['ro']['initialsize']))
    print('   maxidle    : {}'.format(doc['ecommerce']['db']['ro']['maxidle']))
    print('   minidle    : {}'.format(doc['ecommerce']['db']['ro']['minidle']))
    print('   maxtotal   : {}'.format(doc['ecommerce']['db']['ro']['maxtotal']))
    print('rw initialsize: {}'.format(doc['ecommerce']['db']['rw']['initialsize']))
    print('   maxidle    : {}'.format(doc['ecommerce']['db']['rw']['maxidle']))
    print('   minidle    : {}'.format(doc['ecommerce']['db']['rw']['minidle']))
    print('   maxtotal   : {}'.format(doc['ecommerce']['db']['rw']['maxtotal']))
    print('threads       : {}'.format(doc['ecommerce']['executorservice']['threads']))
    print('max threads   : {}'.format(doc['ecommerce']['executorservice']['maxthreads']))
    fh.close()
